# Домашнее задание-6 по теме «Python для анализа данных: pandas» (к лекции от 13.12.2018)

In [121]:
import numpy as np
import pandas as pd

# Задание-1
'''Используем файл keywords.csv.

Необходимо написать гео-классификатор, 
который каждой строке сможет выставить географическую принадлежность определенному региону. 
Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. 
Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток: geo_data

Результат классификации запишите в отдельный столбец region.'''

In [122]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [123]:
df_keywords = pd.read_csv('keywords.csv')
df_keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [124]:
df_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
keyword    100000 non-null object
shows      100000 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [125]:
# Города без последней буквы, чтобы учесть все падежи в запросах ("Москв": содержится в "Москва", "Москве", "Москву"...)
geo_data_no_case = {}
for key, value in geo_data.items():
    geo_data_no_case[key] = []
    for word in value:
        geo_data_no_case[key].append( word[:-1] )
geo_data_no_case

{'Центр': ['москв', 'тул', 'ярославл'],
 'Северо-Запад': ['петербур', 'пско', 'мурманс'],
 'Дальний Восток': ['владивосто', 'сахали', 'хабаровс']}

In [126]:
def get_region_location(x):
    ''' Функция ищет название города в запросе. 
    Если нашла, возвращает регион, иначе - 'undefined' '''
    lst_words = x.split(' ')
    for word in lst_words:
        for key, value in geo_data_no_case.items():
            for v in value:
                if v in word:
                    return key
    
    return 'undefined'

In [127]:
df_keywords['region'] = df_keywords['keyword'].apply(get_region_location)
df_keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [128]:
df_key_gr = df_keywords.groupby('region').count()
df_key_gr 

,keyword,shows
region,,
undefined,98783,98783
Дальний Восток,94,94
Северо-Запад,266,266
Центр,857,857


In [129]:
df_keywords[ df_keywords['region'] == 'Центр' ].head()

,keyword,shows,region
127,авито москва,979292,Центр
143,эхо москвы,889657,Центр
197,погода в москве,745745,Центр
414,погода в москве на 14 дней,400914,Центр
512,карта метро москвы,331857,Центр


# Задание-2
'''Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class'''

In [130]:
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [131]:
def get_ratings_class(x, p_low_rating=2, p_middle_rating=4):
    '''Возвращает оценку словами'''
    if x['rating'] <= p_low_rating:
        return 'низкий рейтинг'
    elif x['rating'] <= p_middle_rating:
        return 'средний рейтинг'
    
    return 'высокий рейтинг' 

In [132]:
df_ratings['class'] = df_ratings.apply(get_ratings_class, p_low_rating=2, p_middle_rating=4, axis=1)
df_ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


# Задание-3
'''Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). 
Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. 
Ответ дайте в днях.'''

In [133]:
df_rating_grouped = df_ratings.groupby('userId').agg( {'timestamp': ['count', 'min', 'max'] } )
df_rating_grouped.columns = ['count', 'min', 'max']
df_rating_grouped.head()

,count,min,max
userId,,,
1,20,1260759108,1260759205
2,76,835355395,835356246
3,51,1298861589,1298932787
4,204,949778714,949982274
5,100,1163373044,1163375145


In [134]:
df_rating_active_users = df_rating_grouped[ df_rating_grouped['count'] >= 100 ]
df_rating_active_users.head()

,count,min,max
userId,,,
4,204,949778714,949982274
5,100,1163373044,1163375145
8,116,1154389340,1154474527
15,1700,997937239,1469330735
17,363,1127468587,1127476640


In [135]:
df_rating_active_users['diff'] = df_rating_active_users['max'] - df_rating_active_users['min']
df_rating_active_users.head()

/home/karaks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,count,min,max,diff
userId,,,,
4,204,949778714,949982274,203560
5,100,1163373044,1163375145,2101
8,116,1154389340,1154474527,85187
15,1700,997937239,1469330735,471393496
17,363,1127468587,1127476640,8053


In [136]:
# Lifetime - разница между максимальным и минимальным значением timestamp для каждого пользователя 
# Cреднее значение Lifetime киноманов:
Lifetime = df_rating_active_users['diff'].mean()
print(Lifetime)

39331748.56653992


In [137]:
# Cреднее значение Lifetime киноманов в днях (1 день = 24 * 60 * 60 секунд )
Lifetime_days = Lifetime / 24 / 60 / 60
print('Cреднее значение Lifetime =', '{:.0f}'.format(Lifetime_days), 'дней')

Cреднее значение Lifetime = 455 дней


# Задание-4
'''
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". 
Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. 
Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, 
а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. 
Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' 
и отсортируйте результат по убыванию рейтинга
'''

In [152]:
years = [x for x in range(1950, 2011)]

In [153]:
def production_year(x, default_year=1900):
    '''Извлекает из строки х и возвращает год'''
    for year in years:
        if str(year) in x['title']:
            return year
    return default_year

In [154]:
df_movies = pd.read_csv('ml-latest-small/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [155]:
df_movies['year'] = df_movies.apply(production_year, default_year=1900, axis=1)
df_movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [156]:
df_joined = pd.merge(df_ratings, df_movies, how='inner', left_on='movieId', right_on='movieId')
df_joined.head()

,userId,movieId,rating,timestamp,class,title,genres,year
0,1,31,2.5,1260759144,средний рейтинг,Dangerous Minds (1995),Drama,1995
1,7,31,3.0,851868750,средний рейтинг,Dangerous Minds (1995),Drama,1995
2,31,31,4.0,1273541953,средний рейтинг,Dangerous Minds (1995),Drama,1995
3,32,31,4.0,834828440,средний рейтинг,Dangerous Minds (1995),Drama,1995
4,36,31,3.0,847057202,средний рейтинг,Dangerous Minds (1995),Drama,1995


In [191]:
df_total = df_joined.groupby('year').mean()[['rating']].reset_index().sort_values(by='rating', ascending=False)
df_total.head(10)

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
25,1974,3.978704
13,1962,3.952446
1,1950,3.915254
28,1977,3.905786
15,1964,3.841492


In [192]:
# Похоже, чем старше фильм, тем больше рейтинг - это значимо
df_total.corr()

,year,rating
year,1.000000,-0.688072
rating,-0.688072,1.000000
